In [1]:
from syllablecounter import load_model, get_data
import numpy as np

In [2]:
def printmd(string, size = 3):
    from IPython.display import Markdown, display
    display(Markdown(f'<font size = {size}>{string}</font>'))
def green(string, deg = 1):
    return f'<span style = "background-color:hsl(125, {deg * 100}%, 50%);">{string}</span>'
def red(string, deg = 1):
    return f'<span style = "background-color:hsl(0, {deg * 100}%, 50%);">{string}</span>'
def ctext(string, deg = 0.5):
    deg = 2 * deg - 1
    if deg > 0:
        return green(string, deg)
    else:
        return red(string, -deg)

In [3]:
counter = load_model()
counter

SyllableCounter(
  (embed): Embedding(30, 64)
  (rnn): GRU(64, 128, num_layers=3, dropout=0.1, bidirectional=True)
  (drops): ModuleList(
    (0): Dropout(p=0.5, inplace=False)
  )
  (norms): ModuleList(
    (0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lins): ModuleList(
    (0): Linear(in_features=256, out_features=256, bias=True)
  )
  (out): Linear(in_features=256, out_features=1, bias=True)
)

In [4]:
import pandas as pd
df = pd.read_csv('data/gutsyls.tsv', sep = '\t')
df.head()

,words,syl_seq,syls
0,nondistillation,100100100101000,5
1,transitoriness,10001010101000,5
2,bipyramidal,10101001100,5
3,nonciteable,10010001100,4
4,livebearer,1000100010,3


In [5]:
hard_words = {'words': [], 'syl_seq': [], 'syls': []}
for idx in np.random.choice(range(len(df)), size = 1000):
    yhat = counter.predict(df['words'][idx], return_confidence = True, return_sequence = True)
    if yhat['num_syls'] != df['syls'][idx]:
        hard_words['words'].append(df['words'][idx])
        hard_words['syl_seq'].append(df['syl_seq'][idx])
        hard_words['syls'].append(df['syls'][idx])
        print(df['words'][idx], df['syl_seq'][idx], df['syls'][idx], yhat['num_syls'], yhat['confidence'])
        print(yhat['probs'])

ath 100 2 1 0.88
[('a', 0.9), ('t', 0.01), ('h', 0.0)]
paleographer 101010101000 5 4 0.39
[('p', 0.9), ('a', 0.0), ('l', 0.34), ('e', 0.04), ('o', 0.9), ('g', 0.01), ('r', 0.89), ('a', 0.0), ('p', 0.89), ('h', 0.0), ('e', 0.01), ('r', 0.0)]
ironsides 110010000 3 4 0.53
[('i', 0.9), ('r', 0.9), ('o', 0.0), ('n', 0.0), ('s', 0.89), ('i', 0.01), ('d', 0.82), ('e', 0.09), ('s', 0.0)]
nauseousness 100100001000 3 4 0.64
[('n', 0.9), ('a', 0.0), ('u', 0.0), ('s', 0.9), ('e', 0.01), ('o', 0.9), ('u', 0.0), ('s', 0.0), ('n', 0.9), ('e', 0.0), ('s', 0.0), ('s', 0.0)]
ursprache 101000100 3 2 0.73
[('u', 0.9), ('r', 0.0), ('s', 0.88), ('p', 0.01), ('r', 0.0), ('a', 0.0), ('c', 0.05), ('h', 0.0), ('e', 0.01)]
faery 10110 3 2 0.26
[('f', 0.89), ('a', 0.0), ('e', 0.21), ('r', 0.56), ('y', 0.34)]
odonovan 10001100 3 4 0.46
[('o', 0.89), ('d', 0.87), ('o', 0.03), ('n', 0.11), ('o', 0.8), ('v', 0.88), ('a', 0.03), ('n', 0.0)]
nkrumah 1000100 2 3 0.65
[('n', 0.9), ('k', 0.88), ('r', 0.02), ('u', 0.0), ('

In [ ]:
while True:
    words = input()
    outputs = counter.predict(words, return_sequence = True, pred_threshold = 0.5)
    string = ''.join([ctext(f'{char}', deg) for char, deg in outputs['probs']])
    printmd(f'{string}&nbsp;= {outputs["num_syls"]} syllables', size = 5)

a


<font size = 5><span style = "background-color:hsl(125, 77.99999713897705%, 50%);">a</span>&nbsp;= 1 syllables</font>